In [15]:
from IPython.display import display, HTML
import pandas as pd
import re

# Explore inventories and explode obvious entries

## Read database with clean dates

In [16]:
df = pd.read_excel("./torzskonyv_clean_dates.xlsx")

display(HTML("<h2>Sheet with cleaned year</h2>"))
display(df)

,Összes darab,Törzskönyvi szám,Megszerzés ideje,Leltárcsoport,year
0,48.0,1874/0001,1874,"2209-2211 Amerika gyűjtemény, 2212-2215 Törlés...",1874.0
1,1456.0,1874/0002,1874,"1-10 Ázsia gyűjtemény, 19-84 Ázsia gyűjtemény,...",1874.0
2,283.0,1874/0003,1874,"11-18 Ázsia gyűjtemény, 158-169 Ázsia gyűjtemé...",1874.0
3,45.0,1874/0004,1874,"235-240 Ázsia gyűjtemény, 332-334 Ázsia gyűjte...",1874.0
4,1.0,1874/0005,1874,170 Ázsia gyűjtemény,1874.0
...,...,...,...,...,...
11766,59.0,2021/0014,NaN,"2021.20.1-44 Ázsia gyűjtemény, 2021.21.1-2 Eur...",NaN
11767,1.0,2021/0015,2005,2021.23.1 Bútor gyűjtemény,2005.0
11768,2.0,2021/0016,2021-03-05 00:00:00,??? Halászat gyűjtemény,2021.0
11769,3.0,2021/0017,NaN,??? Bútor gyűjtemény,NaN


## Read inventory names from text file

In [17]:
from pathlib import Path
def read_inventories(filename:str):
    assert Path(filename).exists()
    def lines(filename):
        with open(filename, encoding="utf8") as f:
            for line in f:
                line = line.strip()
                if line:
                    yield line

    for l in lines(filename):
        yield l

from IPython.display import display, HTML
inventories = sorted([l for l in read_inventories("elnevezesek.txt")])

display(HTML("<h2>Inventories</h2>"))
print(", ".join(inventories))

Adattár, Afrika gyűjtemény, Amerika gyűjtemény, Bútor gyűjtemény, Egyház gyűjtemény, Egyházi gyűjtemény, Európa gyűjtemény, Fényképgyűjtemény, Földművelés gyűjtemény, Gyűjtögetés gyűjtemény, Hagyomány gyűjtemény, Halászat gyűjtemény, Hangszer, Hangszer Népzene gyűjtemény, Hangszer gyűjtemény, Hangszer gyűjtemény, Hangszer gyűjtemény (Népzene oszt.), Hangszer gyűjtemény (Népzene), Indonézia gyűjtemény, Kerámia gyűjtemény, Kézirat-gyűjtemény, Közlekedés gyűjtemény, Mesterség gyűjtemény, Nyomat gyűjtemény, Nyomatgyűjtemény, Népzene (Hangszer) gyűjtemény, Népzene gyűjtemény, Népzene gyűjtemény (Hangszer), Rajz és Nyomat gyűjtemény, Rajz- és nyomatgyűjtemény, Rajzgyűjtemény, Szokás gyűjtemény, Szokás hagyomány, Szokás és játékgyűjtem, Textil gyűjtemény, Textil- és viseletgy., Textilgyűjtemény, Táplálkozás gyűjtemény, Táplálkozás gyűjtemény, Állattartás gyűjtemény, Állattartás-pásztorm., Átány gyűjtemény, Ázsia gyűjtemény, Építkezés gyűjtemény, Óceánia gyűjtemény


## Generate regexp Patterns for each inventory

In [18]:
patterns = [
    *["\d+?-\d+? {inventory}".format(inventory=inventory) for inventory in inventories],
    *["\d+? {inventory}".format(inventory=inventory) for inventory in inventories],
    *["\d+?\.\d+?\.\d+?\.\d+?-\d+? {inventory}".format(inventory=inventory) for inventory in inventories], #"2021.4.1.1-2", 2021.4.1.1-2 Egyházi gyűjtemény
    *["\d+?\.\d+?\.\d+?-\d+? {inventory}".format(inventory=inventory) for inventory in inventories], 
    *["\d+?\.\d+?\.\d+? {inventory}".format(inventory=inventory) for inventory in inventories],
    
    
    "F \d+? Fényképgyűjtemény", # F 71857 Rajzgyűjtemény
    
    "F \d+?-\d+? Fényképgyűjtemény", # F 71611-71644 Fényképgyűjtemény
    "\d+?-\d+? \([F\d\s\-\,]+?\) Fényképgyűjtemény", # 43090-43100 (F4512-F4522) Fényképgyűjtemény
    "\d+? \(F\d+?\) Fényképgyűjtemény",

    "\d+?-\d+? \([R\d\s\-,]+?\) Rajzgyűjtemény" #"42936-43059 (R10583-R10583) Rajzgyűjtemény
    "\d+? \([R\d\s]+?\) Rajzgyűjtemény" # 97339 (R10530) Rajzgyűjtemény
    "F \d+? Rajzgyűjtemény", # F 71857 Rajzgyűjtemény

    "\d+?-\d+? \([F\d\s\-]+?\) Adattár" # 4776-4777 (F 2881-F 2882) Adattár
]

# show
display(HTML("<h2>Patterns</h2>"))
print("\n".join(patterns[:10])+"\n...")

\d+?-\d+? Adattár
\d+?-\d+? Afrika gyűjtemény
\d+?-\d+? Amerika gyűjtemény
\d+?-\d+? Bútor gyűjtemény
\d+?-\d+? Egyház gyűjtemény
\d+?-\d+? Egyházi gyűjtemény
\d+?-\d+? Európa gyűjtemény
\d+?-\d+? Fényképgyűjtemény
\d+?-\d+? Földművelés gyűjtemény
\d+?-\d+? Gyűjtögetés gyűjtemény
...


## Extract all matches from Inventory column by patterns

In [19]:
from typing import List, Tuple

def extract_all_match(text: str, patterns:List[str])->Tuple[List['str'], 'str']:
    if not isinstance(text, str):
        return text
    
    pattern = re.compile( "|".join( ["{}".format(p) for p in patterns] ), re.I )

    matches = [m for m in pattern.finditer(text)]
    if not len(matches):
        return text

    results = []
    leftover = text
    for match in reversed(matches):
        results.append( text[match.start():match.end()] )
        leftover = leftover[0:match.start()] + "[...]" + leftover[match.end():-1]

    results = [res for res in reversed(results)]
    # if re.compile(".*[a-zA-Z].*").match(leftover):
    results.append(leftover)

    return results
    
def last_item(lst: list):
    if isinstance(lst, list) and len(lst)>0:
        return lst[-1]
    else:
        return lst

def test_extraction():
    column = "Leltárcsoport"

    cell = df[column][0]
    results = extract_all_match(cell, patterns)

    assert "2209-2211 Amerika gyűjtemény" in results, results
    assert "2222 Amerika gyűjtemény" in results
    assert "2256-2258 Óceánia gyűjtemény" in results
    assert "4002 Óceánia gyűjtemény" in results
    
    cell = df[column][176]
    results = extract_all_match(cell, patterns)
    assert "10946-10947 Gyűjtögetés gyűjtemény" in results

    cell = df[column][177]
    results = extract_all_match(cell, patterns)
    assert "12818-12843 (F1259-F1274, F5219-F5228) Fényképgyűjtemény" in results

    cell = df[column][11579]
    results = extract_all_match(cell, patterns)
    assert "2019.27.6.1-2 Textil gyűjtemény" in results
    assert "2019.27.5 Amerika gyűjtemény" in results

test_extraction()

# do whole column
entry_series = df["Leltárcsoport"].apply(lambda cell:extract_all_match(cell, patterns))

In [20]:
# explode inventories
df2 = df.copy()
df2['entry'] = entry_series
df2 = df2[['entry', 'year', 'Megszerzés ideje']].explode(column='entry')
df2 = df2.reset_index()


# Mark cleaned inventory rows

In [21]:
def is_exact_match(text: str, patterns:List[str])->List[bool]:
    if not isinstance(text, str):
        return False
    pattern = re.compile( "|".join( ["^{}$".format(p) for p in patterns] ), re.I )
    matches = [m for m in pattern.findall(text)]
    return len(matches)>0


df2['ok'] = df2['entry'].apply(lambda cell: is_exact_match(cell, patterns))
df2['drop'] = df2['entry'].apply(lambda cell: not re.compile(".*[a-zA-Z0-9].*").match(cell) if isinstance(cell, str) else False)
df2['ok'] = df2['ok'] | df2['drop']

print("parsed: {:.0f}%, {} from {}".format(sum(df2['ok'])/len(df2)*100, sum(df2['ok']), len(df2)))

parsed: 95%, 36388 from 38463


## Extract inventory name

In [22]:
def extract_inventory_name(entry):
    if isinstance(entry, str):
        for name in inventories:
            if name.lower() in entry.lower():
                return name
    
    return pd.NA


df2['inventory_name'] = df2[df2['ok']==True]['entry'].apply(extract_inventory_name)

# rebrand inventory names
rebrand = {
    'Textil- és viseletgy.': "Textil gyűjtemény",
    "Textilgyűjtemény": "Textil gyűjtemény",

    "Egyház gyűjtemény": "Egyházi gyűjtemény",

    "Nyomatgyűjtemény": "Nyomat gyűjtemény"

    "Rajz- és nyomatgyűjtemény": "Rajz és Nyomat gyűjtemény"

    "Hangszer": "Hangszer gyűjtemény",
    "Népzene gyűjtemény": "Hangszer gyűjtemény",
    "Népzene gyűjtemény (Hangszer)": "Hangszer gyűjtemény",
    "Népzene (Hangszer) gyűjtemény": "Hangszer gyűjtemény",
    "Hangszer gyűjtemény (Népzene oszt.)": "Hangszer gyűjtemény",
    "Hangszer gyűjtemény (Népzene)": "Hangszer gyűjtemény",
    "Hangszer gyűjtemény": "Hangszer gyűjtemény",
    "Hangszer Népzene gyűjtemény": "Hangszer gyűjtemény",

    "Állattartás-pásztorm.": "Állattartás gyűjtemény"
}
df2['inventory_name'] = df2['inventory_name'].apply(lambda cell: rebrand.get(cell, cell))

In [23]:
df2.to_excel("./torzskonyv_explode.xlsx", index=False)
pd.read_excel("./torzskonyv_explode.xlsx")

,index,entry,year,Megszerzés ideje,ok,drop,inventory_name
0,0,2209-2211 Amerika gyűjtemény,1874.0,1874,True,False,Amerika gyűjtemény
1,0,2216-2217 Amerika gyűjtemény,1874.0,1874,True,False,Amerika gyűjtemény
2,0,2222 Amerika gyűjtemény,1874.0,1874,True,False,Amerika gyűjtemény
3,0,2223-2253 Amerika gyűjtemény,1874.0,1874,True,False,Amerika gyűjtemény
4,0,2256-2258 Óceánia gyűjtemény,1874.0,1874,True,False,Óceánia gyűjtemény
...,...,...,...,...,...,...,...
38458,11767,2021.23.1 Bútor gyűjtemény,2005.0,2005,True,False,Bútor gyűjtemény
38459,11767,[...],2005.0,2005,True,True,NaN
38460,11768,??? Halászat gyűjtemény,2021.0,2021-03-05 00:00:00,False,False,NaN
38461,11769,??? Bútor gyűjtemény,NaN,NaN,False,False,NaN


# JUNK